## 로그 분석을 통한 보안 위험도 예측 AI 경진대회

In [1]:
import pandas as pd
import numpy as np
import os 
import pandas as pds
from dask import dataframe
import re
import numpy as np
import seaborn as sbn
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet

In [2]:
train = pd.read_csv("D:/Desktop/2022-1/EURON/로그분석 보안위험도 필사/train.csv")
test= pd.read_csv("D:/Desktop/2022-1/EURON/로그분석 보안위험도 필사/test.csv")

In [3]:
# EDA

In [4]:
# 1. 다양한 기호 및 숫자 제거
lit = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
def mask(tt):
    tt=tt.apply(lambda x: re.sub(r'(\\n)',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[^a-zA-Zㄱ-ㅣ가-힣0-9:=\s\(\)./,\<\>]+',' ',x))
    #tt=tt.apply(lambda x: re.sub(r' ?(?P<note>[:=\(\)./,\<\>]) ?', ' \g<note> ', x))
    tt=tt.apply(lambda x: re.sub(r'[0-9]+',' ',x))
    tt=tt.apply(lambda x: re.sub(r"':/()",' ',x))
    tt=tt.apply(lambda x: re.sub(r':',' ',x))
    tt=tt.apply(lambda x: re.sub(r',',' ',x))
    # = tt.apply(lambda x: re.sub(r'(',' ',x))
    #t = tt.apply(lambda x: re.sub(r')',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',x))
    for st in lit:
        st = " "+st + " "
        tt=tt.apply(lambda x: re.sub(st,' ',x))
    tt=tt.apply(lambda x: re.sub(r'\s+',' ',x))
    
    return tt

In [5]:
train['pre_log'] = mask(train.full_log)
test['pre_log'] = mask(test.full_log)

In [6]:
# 2. 영어 단어가 아니거나 3글자 미만인지인 경우에는 삭제

In [7]:
#포함되어 있는 단어 모음 생성
def count_word(data):
    tem = list(data['pre_log'].str.split(" "))
    all_word = []
    for word in tem:
        all_word.extend(word)
    words = pd.Series(all_word)
    return words.value_counts()

In [8]:
train_count = count_word(train)
test_count = count_word(test)

In [9]:
train_words = list(set(train_count.index))
test_words = list(set(test_count.index))

In [10]:
#영어 단어 + 3글자 이상인 경우만 True
from nltk.corpus import words
def check_words(word_list: list):
    re = [False] * len(word_list)
    for i,word in enumerate(word_list):
        if len(word) < 3:
            continue
        word = word.lower()
        if word in words.words():
            re[i] = True
    return re

word_list = ['Promiscuous', 'ab', 'nihongo', 'abstract', 'pedo','gid']

In [11]:
#check_words는 단어인지를 확인해주고 결과를 t/f로 반환
check_words(word_list)

[True, False, False, True, False, True]

In [12]:
train_tf = check_words(train_words)
test_tf = check_words(test_words)

In [13]:
#단어인지 확인해주는 딕셔너리 만들어주기 (train_isword,test_isword)
train_isword = dict()
test_isword = dict()

total_words = list(train_words)
test_words = list(test_words)

for i in range(len(train_words)):
    train_isword[train_words[i].lower()] = train_tf[i]
    
for i in range(len(test_words)):
    test_isword[test_words[i].lower()] = test_tf[i]

In [14]:
import copy

def checking_train(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if train_isword[word]:
            re[i] = True
    return re


def checking_test(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if test_isword[word]:
            re[i] = True
    return re



def cutt_train(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_train(splited)
    c = np.array(splited)
    real_words = list(c[check])    
    tem = " ".join(real_words)
    #tem = tem.lower() 
    return tem

def cutt_test(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_test(splited)
    c = np.array(splited)
    real_words = list(c[check])  
    tem = " ".join(real_words)
    #tem = tem.lower()
    return tem

In [15]:
train['cut'] = train['pre_log'].map(cutt_train)
test['cut'] = test['pre_log'].map(cutt_test)

In [16]:
del train['full_log']
del test['full_log']

del train['pre_log']
del test['pre_log']

In [17]:
# 4. 나중에 사용될 inornot 리스트

In [18]:
train_bag = train['cut'].unique()
len(train_bag)

6544

In [19]:
#inornot을 통해서 train안에 같은 문장이 있으면 0 없으면 1
from tqdm import tqdm
inornot = np.zeros(len(test))
for i in tqdm(range(len(test))):
    tem = test.iloc[i]['cut']
    if tem not in train_bag:
        inornot[i] = 1

100%|███████████████████████████████████████████████████████████████████████| 1418916/1418916 [25:02<00:00, 944.26it/s]


In [20]:
# 모델

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

#남은 단어가 없는 경우에는 'missing' 으로 대체해주었습니다.
train['cut'] = train['cut'].replace('','missing',regex=True)
test['cut'] = test['cut'].replace('','missing',regex=True)



train = train.fillna("missing")
test = test.fillna("missing")

train

,id,level,cut
0,0,0,type error warning collection level error erro...
1,1,0,action with response code type unavailable exc...
2,2,0,type error warning collection level error erro...
3,3,0,type error warning collection level error erro...
4,4,1,type audit arch success yes exit gid suid none...
...,...,...,...
472967,472967,0,error
472968,472968,1,type audit arch success yes exit gid suid none...
472969,472969,0,type log error task manager message poll for w...
472970,472970,0,type error warning collection level error erro...


In [22]:
train_text=list(train['cut'])
train_level=np.array(train['level'])

test_text=list(test['cut'])

In [23]:
vectorizer=CountVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [24]:
forest=RandomForestClassifier(n_estimators=100,random_state = 1 )
forest.fit(train_features,train_level)

RandomForestClassifier(random_state=1)

In [25]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

In [26]:
#Validation을 통한 threshold 조절
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 1))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 2))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.95) & (results == 3))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 4))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 5))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 6))[0]]=7

In [27]:
#Validation을 통한 threshold 조절
results[np.where((np.max(results_proba, axis=1)<0.94001) & (np.max(results_proba, axis=1) > 0.93999) & (results == 1))[0]]=7 
results[np.where((np.max(results_proba, axis=1)<0.611657) & (np.max(results_proba, axis=1) > 0.6116568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.571657) & (np.max(results_proba, axis=1) > 0.5716568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.68001) & (np.max(results_proba, axis=1) > 0.67999) & (results == 5))[0]]=7 

In [28]:
# 결과저장

In [30]:
submission=pd.read_csv('D:/Desktop/2022-1/EURON/로그분석 보안위험도 필사/sample_submission.csv')
submission['level']=results
submission['level'].value_counts()

0    1002369
1     396443
3      12813
5       6418
7        780
2         34
4         34
6         25
Name: level, dtype: int64

In [31]:
submission.to_csv("tem_answer.csv",index = False)

In [32]:
# 추가작업

In [33]:
submission['inornot'] = pd.Series(inornot)

cal = submission[(submission['level']==7) & (submission['inornot'] == 0)].index
not7_id = list(test.iloc[cal]['id'])
pd.Series(not7_id).to_csv("not7_id.csv",index= False)

## COVID-19 Literature Clustering